This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [77]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [78]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "/kaggle/input/muffin-vs-chihuahua-image-classification/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "/kaggle/input/muffin-vs-chihuahua-image-classification/test"    # e.g. './muffin-vs-chihuahua/test'

In [79]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [80]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [81]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [82]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [83]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 91s 747ms/step - accuracy: 0.5815 - loss: 1.0407 - val_accuracy: 0.8011 - val_loss: 0.6677
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 736ms/step - accuracy: 0.7793 - loss: 0.6110 - val_accuracy: 0.7534 - val_loss: 0.6180
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 740ms/step - accuracy: 0.7964 - loss: 0.5198 - val_accuracy: 0.7577 - val_loss: 0.5755
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 736ms/step - accuracy: 0.8167 - loss: 0.4884 - val_accuracy: 0.8423 - val_loss: 0.4120
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 738ms/step - accuracy: 0.8371 - loss: 0.4444 - val_accuracy: 0.8307 - val_loss: 0.4488
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 740ms/step - accuracy: 0.8278 - loss: 0.4527 - val_accuracy: 0.8635 - val_loss: 0.4012
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 88s 739ms/step - accuracy: 0.8506 - loss: 0.4130 - val_accuracy: 0.8942 - val_loss: 0.3201
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 87s 733ms/step - accuracy: 0.8618 - loss: 0

In [84]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 315ms/step - accuracy: 0.8054 - loss: 0.5042
Test Accuracy: 0.8513513803482056


In [85]:
# SAVE THE MODEL
model.save('/kaggle/working/models/exercise_6_trained_model_improved.h5')

In [88]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='/kaggle/working/models/exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")
    return pred


In [89]:
# Example usage:
pred1 = predict_image("/kaggle/input/muffin-vs-chihuahua-image-classification/test/chihuahua/img_0_1071.jpg")
pred2 = predict_image("/kaggle/input/muffin-vs-chihuahua-image-classification/test/muffin/img_0_105.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


Prediction: Muffin (confidence: 0.31)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Prediction: Chihuahua (confidence: 0.88)


### Accuracy
Test Accuracy = 85.13%

### Prediction and Confidence
Run1:<br>
    Prediction = incorrect<br>
    Confidence = 0.31<br><br>
Run2:<br>
    Prediction = incorrect<br>
    Confidence = 0.88<br>